In [14]:
import pandas as pd
import tensorflow as tf
import numpy as np
import datetime as dt
import cv2
import os
from collections import Counter
from tqdm import tqdm

ModuleNotFoundError: No module named 'tensorflow'

# 图像预处理

## 图像读取

In [13]:
# 定义图像处理函数
def read_img(path):
    print("数据集地址："+path)
    imgs = []
    labels = []
    for root, dirs, files in tqdm(os.walk(path)):
        for file in files:
            # print(path+'/'+file+'/'+folder)
            # 读取的图片
            img = cv2.imread(os.path.join(root, file))
            # 将读取的图片数据加载到imgs[]列表中
            imgs.append(img)
            # 将图片的label加载到labels[]中，与上方的imgs索引对应
            labels.append(str(os.path.basename(root)))
    return imgs,labels

In [12]:
img_train = 'image/train/'
img_test = 'image/test/'
# 调用读取图片的函数，得到图片和labels的数据集
x_train_list, y_train_list = read_img(img_train)
print("完成训练图片读取！")
x_test_list, y_test_list = read_img(img_test)
print("完成测试图片读取！")

数据集地址：image/train/


NameError: name 'tqdm' is not defined

## 类别编码+归一化处理

In [5]:
# 水果类别编码函数
def change_label(label,path):
    from sklearn.preprocessing import LabelEncoder
    le_credit_level = LabelEncoder().fit(label)
    new_label = le_credit_level.transform(label)
    label_counts = len(Counter(label))
    # 保存编码对应关系
    lb_list = le_credit_level.classes_
    print("编码后类别个数:",label_counts)
    print("编码后对应关系:",lb_list)
    print("编码后类标数量:",Counter(new_label))
    np.save(path, lb_list)
    return new_label

In [11]:
# 类别编码
path = 'data/trainlabel_list.npy'
y_train = change_label(y_train_list,path)
path = 'data/testlabel_list.npy'
y_test = change_label(y_test_list,path)
print("完成编码！")
print("*"*10)
# 将图像数据转换为numpy数组
x_train_np,y_train_np = np.asarray(x_train_list, np.float32),np.asarray(y_train, np.int32)
x_test_np,y_test_np =  np.asarray(x_test_list, np.float32),np.asarray(y_test, np.int32)
# 归一化
x_train_np = x_train_np/255.0
x_test_np = x_test_np/255.0
print("处理后训练数据：",x_train_np.shape)
print("处理后测试数据：",x_test_np.shape)

NameError: name 'y_train_list' is not defined

# TFRecord存储

In [8]:
# np转为TFRecord格式存储
def save_tfrecords(data, label, desfile):
    with tf.io.TFRecordWriter(desfile) as writer:
        for i in range(len(data)):
            features = tf.train.Features(
                feature = {
                    "data":tf.train.Feature(bytes_list = tf.train.BytesList(value = [bytes(data[i])] )),
                    "label":tf.train.Feature(int64_list = tf.train.Int64List(value = [label[i]])),
#                     "shape": tf.train.Feature(int64_list =tf.train.Int64List(value=[data[i].shape[0],data[i].shape[1],data[i].shape[2]]))
                }
            )
            example = tf.train.Example(features = features)
            serialized = example.SerializeToString()
            writer.write(serialized)
    writer.close()
    print("tfrecord执行成功，数据保存地址：",desfile)

In [9]:
# 保存训练和测试数据
save_tfrecords(x_train_np, y_train_np, "data/train_data.tfrecords")
save_tfrecords(x_test_np, y_test_np, "data/test_data.tfrecords")

NameError: name 'x_train_np' is not defined